In [24]:
import transformers
transformers.logging.set_verbosity_warning()

In [ ]:
# load TransformerText.txt
with open('./Data/TransformerText.txt', 'r') as file:
    text = file.read()

In [ ]:
# Tokenize a text using the autotokenizer of the transformers library

# Import the AutoTokenizer class
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

In [ ]:
# Tokenize the text
tokenized_text = tokenizer.tokenize(text)

# Print the tokenized text
print(tokenized_text)

In [ ]:
tokenizer(text)

In [1]:
# load Train.csv and Test.csv into same datasetdic
import pandas as pd
import numpy as np
from datasets import load_dataset

data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("../Data/IMDBsentiment/IMDBsentiment", data_files=data_files)

c:\Users\monox\anaconda3\envs\transformer_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration IMDBsentiment-0473a1081b52037e
c:\Users\monox\anaconda3\envs\transformer_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Found cached dataset csv (C:/Users/monox/.cache/huggingface/datasets/csv/IMDBsentiment-0473a1081b52037e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 2/2 [00:00<00:00, 13.79it/s]


In [3]:
# remduce dataset train to 100 samples
dataset['train'] = dataset['train'].select(range(100))
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_imdb = dataset.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  8.55ba/s]
Loading cached processed dataset at C:\Users\monox\.cache\huggingface\datasets\csv\IMDBsentiment-0473a1081b52037e\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-a13dd62b552c8163.arrow


In [7]:
tokenized_imdb["train"][:1]

{'text': ['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'],
 'label': [0],
 'input_ids': [[101,
   1045,
   3473,
   2039,
   1006,
   1038,
   1012,
   3551,
   1007,
   

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [13]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\monox\anaconda3\envs\transformer_env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14
  Number of trainable parameters = 66955010
 50%|█████     | 7/14 [03:49<03:19, 28.57s/it]The following columns in the evaluation set don't have a corr

{'eval_loss': 0.6879410743713379, 'eval_accuracy': 0.501, 'eval_runtime': 4116.7849, 'eval_samples_per_second': 1.215, 'eval_steps_per_second': 0.076, 'epoch': 1.0}


Model weights saved in my_awesome_model\checkpoint-7\pytorch_model.bin
tokenizer config file saved in my_awesome_model\checkpoint-7\tokenizer_config.json
Special tokens file saved in my_awesome_model\checkpoint-7\special_tokens_map.json
100%|██████████| 14/14 [1:15:30<00:00, 169.08s/it]  The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
                                                  
100%|██████████| 14/14 [2:08:44<00:00, 169.08s/it]Saving model checkpoint to my_awesome_model\checkpoint-14
Configuration saved in my_awesome_model\checkpoint-14\config.json


{'eval_loss': 0.6841050982475281, 'eval_accuracy': 0.567, 'eval_runtime': 3193.9312, 'eval_samples_per_second': 1.565, 'eval_steps_per_second': 0.098, 'epoch': 2.0}


Model weights saved in my_awesome_model\checkpoint-14\pytorch_model.bin
tokenizer config file saved in my_awesome_model\checkpoint-14\tokenizer_config.json
Special tokens file saved in my_awesome_model\checkpoint-14\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_awesome_model\checkpoint-14 (score: 0.6841050982475281).
100%|██████████| 14/14 [2:08:47<00:00, 551.97s/it]

{'train_runtime': 7727.5342, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.002, 'train_loss': 0.6903149059840611, 'epoch': 2.0}


TrainOutput(global_step=14, training_loss=0.6903149059840611, metrics={'train_runtime': 7727.5342, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.002, 'train_loss': 0.6903149059840611, 'epoch': 2.0})

In [25]:
from transformers import pipeline

text = "This is a very very bad movie."

classifier = pipeline("sentiment-analysis", model="./trained_model/checkpoint-XX/")
print(classifier(text))

[{'label': 'POSITIVE', 'score': 0.5119481682777405}]
